``` sql
create table sharktank_india(
sharks varchar(50),
pitchers varchar(50),
valuation int,
equity int);

insert into sharktank_india values('Piyush','A',270,10);
insert into sharktank_india values('Piyush','B',250,10);
insert into sharktank_india values('Piyush','C',150,30);

insert into sharktank_india values('Anupam','D',100,45);
insert into sharktank_india values('Anupam','E',300,30);
insert into sharktank_india values('Anupam','F',250,40);

insert into sharktank_india values('Ashneer','G',200,18);
insert into sharktank_india values('Ashneer','H',250,20);
insert into sharktank_india values('Ashneer','I',250,30);

insert into sharktank_india values('Aman','J',260,30);
insert into sharktank_india values('Aman','K',340,35);
insert into sharktank_india values('Aman','L',230,28);
insert into sharktank_india values('Aman','M',500,50);

select * from sharktank_india;

+---------+----------+-----------+--------+
| sharks  | pitchers | valuation | equity |
+---------+----------+-----------+--------+
| Piyush  | A        |       270 |     10 |
| Piyush  | B        |       250 |     10 |
| Piyush  | C        |       150 |     30 |
| Anupam  | D        |       100 |     45 |
| Anupam  | E        |       300 |     30 |
| Anupam  | F        |       250 |     40 |
| Ashneer | G        |       200 |     18 |
| Ashneer | H        |       250 |     20 |
| Ashneer | I        |       250 |     30 |
| Aman    | J        |       260 |     30 |
| Aman    | K        |       340 |     35 |
| Aman    | L        |       230 |     28 |
| Aman    | M        |       500 |     50 |
+---------+----------+-----------+--------+
```

Q. Write an SQL query to find the smartest Shark from the below table.
A smart shark is the one whose every deal valuation must be greater than the average valuation of all other sharks deals and similarly, every deal equity must be greater than the average equity of all the other shark's equity.

expected output:
``` sql
+--------+
| sharks |
+--------+
| Aman   |
+--------+
```
---

#### **Solution-1:**
``` sql
WITH cte AS (
  SELECT 'Piyush' AS sharks, AVG(valuation) AS avg_valuation, AVG(equity) AS avg_equity
  FROM sharktank_india WHERE sharks <> 'Piyush'
  UNION ALL
  SELECT 'Anupam', AVG(valuation), AVG(equity) FROM sharktank_india WHERE sharks <> 'Anupam'
  UNION ALL
  SELECT 'Ashneer', AVG(valuation), AVG(equity) FROM sharktank_india WHERE sharks <> 'Ashneer'
  UNION ALL
  SELECT 'Aman', AVG(valuation), AVG(equity) FROM sharktank_india WHERE sharks <> 'Aman'
),
cte2 AS (
  SELECT s.*, COUNT(*) OVER (PARTITION BY s.sharks) AS cnt, c.avg_valuation, c.avg_equity
  FROM sharktank_india s
  JOIN cte c ON s.sharks = c.sharks
)
SELECT sharks
FROM cte2
WHERE valuation > avg_valuation AND equity > avg_equity
GROUP BY sharks, cnt
HAVING COUNT(*) = cnt;
```
----

#### **Solution-2**
``` sql
WITH shark_avg AS (
    SELECT s1.sharks,
           AVG(CASE WHEN s2.sharks <> s1.sharks THEN s2.valuation END) AS other_avg_valuation,
           AVG(CASE WHEN s2.sharks <> s1.sharks THEN s2.equity END) AS other_avg_equity
    FROM sharktank_india s1
    CROSS JOIN sharktank_india s2
    GROUP BY s1.sharks
),
smart_check AS (
    SELECT s.sharks,
           MIN(CASE WHEN s.valuation > sa.other_avg_valuation THEN 1 ELSE 0 END) AS valuation_check,
           MIN(CASE WHEN s.equity > sa.other_avg_equity THEN 1 ELSE 0 END) AS equity_check
    FROM sharktank_india s
    JOIN shark_avg sa
      ON s.sharks = sa.sharks
    GROUP BY s.sharks
)
SELECT sharks
FROM smart_check
WHERE valuation_check = 1 AND equity_check = 1;
```
---